# 进化优化算法作业 3：蚁群优化算法解决 TSP 问题
旅行商问题（Traveling Salesman Problem, TSP）是组合优化中的经典问题之一。其内容是给定一组城市及城市之间的距离，要求找出一条最短的路径，使得旅行商从某一城市出发，经过每个城市且仅经过一次，最后回到出发城市。

TSP 的主要难题在于其计算复杂度。随着城市数量的增加，可能的路径组合数呈指数级增长，导致穷举法在实际应用中不可行。因此，寻找高效的近似算法成为解决 TSP 的关键。

蚁群算法（Ant Colony Optimization, ACO）是一种基于自然界蚂蚁觅食行为的启发式算法。蚂蚁通过在路径上留下信息素来指引其他蚂蚁选择路径。ACO 通过模拟这一过程，逐步优化路径选择，最终找到 TSP 的近似最优解。具体来说，蚁群算法通过多次迭代，利用信息素的正反馈机制和路径选择的随机性，逐步收敛到最优或近似最优的路径。

## 详细说明：
本问题中给定 14 个城市的坐标，使用蚁群优化算法或其他优化的启发式算法寻找一条最短的遍历所有城市的路径。

以下是这 14 个城市的坐标：
| 城市编号 | X 坐标 | Y 坐标 | 城市编号 | X 坐标 | Y 坐标 |
|---------|-------|-------|---------|-------|-------|
| 1       | 16.47 | 96.10 | 8       | 17.20 | 96.29 |
| 2       | 16.47 | 94.44 | 9       | 16.30 | 97.38 |
| 3       | 20.09 | 92.54 | 10      | 14.05 | 98.12 |
| 4       | 22.39 | 93.37 | 11      | 16.53 | 97.38 |
| 5       | 25.23 | 97.24 | 12      | 21.52 | 95.59 |
| 6       | 22.00 | 96.05 | 13      | 19.41 | 97.13 |
| 7       | 20.47 | 97.02 | 14      | 20.09 | 92.55 |

## Part A. 问题解决思路简述
蚁群算法（ACO）解决 TSP 问题的思路如下：

1. **初始化**：
    - 初始化信息素矩阵，每条路径上的信息素初始值相同。
    - 设置算法参数，如蚂蚁数量、信息素重要性因子、启发式因子、信息素挥发系数等。

2. **构建解**：
    - 每只蚂蚁从一个随机选择的城市出发。
    - 根据概率选择下一个城市，概率由信息素浓度和启发式信息（如城市间距离的倒数）共同决定。
    - 重复上述步骤，直到蚂蚁遍历完所有城市，形成一条完整路径。

3. **更新信息素**：
    - 根据蚂蚁走过的路径长度更新信息素。路径越短，信息素增加越多。
    - 信息素挥发，防止过早收敛到局部最优解。

4. **迭代**：
    - 重复构建解和更新信息素的过程，直到达到预定的迭代次数或满足其他终止条件。

5. **输出结果**：
    - 记录并输出迭代过程中找到的最优路径及其长度。

## Part B. 算法求解

In [36]:
import numpy as np
import random as rand

In [37]:

city_data = np.array([
    [1, 16.47, 96.10],
    [2, 16.47, 94.44],
    [3, 20.09, 92.54],
    [4, 22.39, 93.37],
    [5, 25.23, 97.24],
    [6, 22.00, 96.05],
    [7, 20.47, 97.02],
    [8, 17.20, 96.29],
    [9, 16.30, 97.38],
    [10, 14.05, 98.12],
    [11, 16.53, 97.38],
    [12, 21.52, 95.59],
    [13, 19.41, 97.13],
    [14, 20.09, 92.55]
])

In [116]:
# 计算城市间距离
def calculate_distance(city_data):
    num_cities = city_data.shape[0]
    distance_matrix = np.zeros((num_cities, num_cities))
    for i in range(num_cities):
        for j in range(num_cities):
            if i != j:
                distance_matrix[i][j] = np.sqrt((city_data[i][1] - city_data[j][1])**2 + (city_data[i][2] - city_data[j][2])**2)
    return distance_matrix

# 初始化信息素矩阵
def initialize_pheromone_matrix(num_cities, initial_pheromone):
    return np.full((num_cities, num_cities), initial_pheromone)

# 选择下一个城市
def select_next_city(pheromone_matrix, distance_matrix, visited, current_city, alpha, beta):
    num_cities = pheromone_matrix.shape[0]
    probabilities = np.zeros(num_cities)
    for i in range(num_cities):
        if i not in visited:
            probabilities[i] = (pheromone_matrix[current_city][i] ** alpha) * ((1.0 / distance_matrix[current_city][i]) ** beta)
    probabilities_sum = probabilities.sum()
    if probabilities_sum == 0:
        return np.random.choice([i for i in range(num_cities) if i not in visited])
    probabilities /= probabilities_sum
    return np.random.choice(range(num_cities), p=probabilities)

# 更新信息素矩阵
def update_pheromone_matrix(pheromone_matrix, all_paths, distance_matrix, evaporation_rate, Q):
    num_cities = pheromone_matrix.shape[0]
    pheromone_matrix *= (1 - evaporation_rate)
    for path, length in all_paths:
        for i in range(num_cities - 1):
            pheromone_matrix[path[i]][path[i + 1]] += Q / length
        pheromone_matrix[path[-1]][path[0]] += Q / length

# 蚁群算法求解 TSP
def ant_colony_optimization(city_data, num_ants, num_iterations, alpha, beta, evaporation_rate, Q, initial_pheromone):
    num_cities = city_data.shape[0]
    distance_matrix = calculate_distance(city_data)
    pheromone_matrix = initialize_pheromone_matrix(num_cities, initial_pheromone)
    
    best_path = None
    best_length = float('inf')
    
    for iteration in range(num_iterations):
        all_paths = []
        for ant in range(num_ants):
            visited = []
            current_city = np.random.randint(num_cities)
            visited.append(current_city)
            
            while len(visited) < num_cities:
                next_city = select_next_city(pheromone_matrix, distance_matrix, visited, current_city, alpha, beta)
                visited.append(next_city)
                current_city = next_city
            
            path_length = sum(distance_matrix[visited[i]][visited[i + 1]] for i in range(num_cities - 1))
            path_length += distance_matrix[visited[-1]][visited[0]]
            all_paths.append((visited, path_length))
            
            if path_length < best_length:
                best_length = path_length
                best_path = visited
        
        update_pheromone_matrix(pheromone_matrix, all_paths, distance_matrix, evaporation_rate, Q)
    
    return best_path, best_length

# 设置参数并运行算法
num_ants = 50
num_iterations = 100
alpha = 1.0
beta = 5.0
evaporation_rate = 0.5
Q = 50
initial_pheromone = 1.0

best_path, best_length = ant_colony_optimization(city_data, num_ants, num_iterations, alpha, beta, evaporation_rate, Q, initial_pheromone)
print("Best path:", best_path)
print("Best path length:", best_length)

Best path: [0, 10, 8, 9, 1, 13, 2, 3, 4, 5, 11, 6, 12, 7]
Best path length: 29.688931283833764
